## <center><strong>Notebook describtion & our <span style= 'color: #51fcc6'>Questions</span>

In this notebook we are going to create the charts that we are going to use in the final pdf report and the tools we are going<br>
to use are matplotlib and seaborn.

#### Questions we are going to awnser:
- How many jobs are available per job title on each platform.
- 
- 
- 

## <center><strong>Importing the <span style= 'color: #48e0dc'>Packeges 

In [ ]:
import os
import warnings
import matplotlib
import pandas             as pd
import numpy              as np
import seaborn            as sns
import matplotlib.pyplot  as plt

from google.cloud         import bigquery
from wordcloud            import WordCloud
from collections          import Counter

In [ ]:
%matplotlib inline
plt.rcParams['font.family'] = 'Candara'

warnings.filterwarnings('ignore')

credentials_path :str = '../credentials.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

FONT             :int  = 17
COLORS           :list = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']
NUMERICS         :list = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
                          'uint16', 'uint32', 'uint64']